In [ ]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 29.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from rdkit import Chem
import re
from tqdm import tqdm
import warnings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
# Load the dataset
df = pd.read_csv('BBBP.csv')

# Calculate chain lengths
df['smiles_length'] = df['smiles'].apply(lambda x: len(x))

atoms = []
for molecule in df['smiles']:
    try:
        mol = Chem.MolFromSmiles(molecule)
        for atom in mol.GetAtoms():
            atoms.append(atom.GetSymbol())
    except:
        pass

unique_atoms = set(atoms)

# Count the frequency of each token
token_counts = Counter(atoms)

In [ ]:
# Create a report
report = f"""
### SMILES Format:
SMILES (Simplified Molecular Input Line Entry System) is a notation that allows a user to represent a chemical structure in a way that can be used by the computer. It consists of a series of symbols and characters representing the structure of a molecule.

### Chain Length:
Chain length refers to the length of the molecular structure represented in the SMILES format. In this context, we can interpret it as the number of characters in the SMILES string.

#### Chain Length Distribution:
{df['smiles_length'].describe()}

### Token Variety and Frequency:
Tokens are individual units (characters or sub-sequences) that make up the SMILES strings. Analyzing the variety and frequency of tokens can provide insights into the diversity of molecular structures in the dataset.

#### Token Variety:
{len(token_counts)} unique tokens

#### Token Frequency:
{token_counts.most_common()}
"""

print(report)


### SMILES Format:
SMILES (Simplified Molecular Input Line Entry System) is a notation that allows a user to represent a chemical structure in a way that can be used by the computer. It consists of a series of symbols and characters representing the structure of a molecule.

### Chain Length:
Chain length refers to the length of the molecular structure represented in the SMILES format. In this context, we can interpret it as the number of characters in the SMILES string.

#### Chain Length Distribution:
count    2050.000000
mean       51.474146
std        30.620659
min         3.000000
25%        33.000000
50%        45.000000
75%        61.000000
max       400.000000
Name: smiles_length, dtype: float64

### Token Variety and Frequency:
Tokens are individual units (characters or sub-sequences) that make up the SMILES strings. Analyzing the variety and frequency of tokens can provide insights into the diversity of molecular structures in the dataset.

#### Token Variety:
13 unique toke

In [ ]:
all_smiles = ''.join(df['smiles'].tolist())
unique_syms_nums = ''.join(char for char in all_smiles if not char.isalpha())
unique_tokens = set(unique_atoms)
unique_tokens.update(set(unique_syms_nums))
unique_tokens = list(unique_tokens) + ['c', 'n', 'o', 's']
print(unique_tokens)

['F', '%', '0', 'N', 'O', '9', '4', 'H', ']', '(', 'P', 'B', '=', '6', '7', 'Br', 'Ca', '[', 'Cl', '2', '.', 'C', '#', '8', '/', '@', '3', ')', 'S', '-', '5', '1', 'I', '\\', '+', 'Na', 'c', 'n', 'o', 's']


In [ ]:
# Encode tokens using one-hot encoding
def smiles_to_onehot(molecule, token_index, code_size):
    molecule_tokens = []
    current_element = ''
    i = 0
    while i < len(molecule):
        current_element += molecule[i]
        while current_element in unique_tokens:
            i += 1
            if i < len(molecule):
                current_element += molecule[i]
            else:
                current_element += 'T'
        molecule_tokens.append(current_element[:-1])
        current_element = ''

    result = torch.zeros(len(molecule_tokens), code_size)
    for i, token in enumerate(molecule_tokens):
        onehot = torch.zeros(code_size)
        onehot[token_index[token]] = 1
        result[i,:] = onehot
    return result

# Convert labels to PyTorch tensor
labels = torch.tensor(df['p_np'].values, dtype=torch.float32)

# Create token index
token_index = {token: i for i, token in enumerate(unique_tokens)}

# Encode SMILES strings with padding
encoded_smiles = [smiles_to_onehot(molecule, token_index, len(token_index)) for molecule in df['smiles']]

max_tokens_in_molecule = max(len(tensor) for tensor in encoded_smiles)
# Zero-pad each tensor to make them all have the same length of token_index
encoded_smiles = [torch.nn.functional.pad(tensor, (0, 0, 0, max_tokens_in_molecule - len(tensor))) for tensor in encoded_smiles]

# Stack the padded tensors to create the final tensor
encoded_smiles = torch.stack(encoded_smiles)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(encoded_smiles, labels, test_size=0.2, random_state=42)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
# Define the FC neural network
class FCNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNet, self).__init__()
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1] * X_train.shape[2]
hidden_size = 128
output_size = 1
model = FCNet(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 50
for epoch in tqdm(range(epochs)):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on test data
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    predictions = (test_outputs.squeeze() > 0.5).float()
    accuracy = (predictions == y_test).float().mean()

print(f"\n Accuracy on test data: {accuracy.item() * 100:.2f}%")


100%|██████████| 50/50 [00:57<00:00,  1.16s/it]


 Accuracy on test data: 86.34%


In [ ]:
# Define the LSTM + FC neural network
class MolecularLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, lstm_layers, fc_hidden_size, output_size):
        super(MolecularLSTM, self).__init__()
        self.num_layers = lstm_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, 10, num_layers=lstm_layers, batch_first=True)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(10*hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(fc_hidden_size)
        self.fc2 = nn.Linear(fc_hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        hidden_states = torch.zeros(self.num_layers, x.size(0), 10)
        cell_states = torch.zeros(self.num_layers, x.size(0), 10)
        x, _ = self.lstm(x, (hidden_states, cell_states))
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(self.bn(x))
        x = self.sigmoid(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[2]
hidden_size = X_train.shape[1]
lstm_layers = 1
fc_hidden_size = 64
output_size = 1

model = MolecularLSTM(input_size, hidden_size, lstm_layers, fc_hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in tqdm(range(epochs)):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on test data
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    predictions = (test_outputs.squeeze() > 0.5).float()
    accuracy = (predictions == y_test).float().mean()

print(f"\n Accuracy on test data: {accuracy.item() * 100:.2f}%")

100%|██████████| 100/100 [03:11<00:00,  1.91s/it]


 Accuracy on test data: 83.17%


In [ ]:
# Define the BiLSTM + FC neural network
class MolecularBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, lstm_layers, fc_hidden_size, output_size):
        super(MolecularLSTM, self).__init__()
        self.num_layers = lstm_layers
        self.hidden_size = hidden_size
        self.bilstm = nn.LSTM(input_size, 10, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(10*hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(fc_hidden_size)
        self.fc2 = nn.Linear(fc_hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        hidden_states = torch.zeros(self.num_layers, x.size(0), 10)
        cell_states = torch.zeros(self.num_layers, x.size(0), 10)
        x, _ = self.lstm(x, (hidden_states, cell_states))
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(self.bn(x))
        x = self.sigmoid(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[2]
hidden_size = X_train.shape[1]
lstm_layers = 1
fc_hidden_size = 64
output_size = 1

model = MolecularLSTM(input_size, hidden_size, lstm_layers, fc_hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in tqdm(range(epochs)):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on test data
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    predictions = (test_outputs.squeeze() > 0.5).float()
    accuracy = (predictions == y_test).float().mean()

print(f"\n Accuracy on test data: {accuracy.item() * 100:.2f}%")

100%|██████████| 100/100 [03:11<00:00,  1.92s/it]


 Accuracy on test data: 82.20%
